# HERMES 3.0 UK Biobank Phenotyping

## Setup
You need to set the project and record IDs below.

* **Project ID**: the `Project ID` is available from the 'Settings' tab after clicking on your project in the UK Biobak RAP.  
* **Record ID**: the record or dataset `ID` is available from the 'Manage' tab, where you select the checkbox to the left of your dispensed `.dataset` (e.g. app81499_20241105095754.dataset) which then displays the `ID` in the information pane on the right. 

In [3]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
recordid  <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"

### Libraries

In [4]:
library(glue)
library(data.table)

## Data extraction
Given the large datasets we make use of the `dx run table-exporter` to extract the required phenotype data. The extraction function below will create a table-exportor job which you will be able to track in the 'Monitor' table on your RAP's homepage. The data will be extracted to your project into a folder called `hermes3_data`. The data is not immediately uploaded to this session, although we will import it later. 

To get the small data dictionaries locally in this session we use the `dx extract_dataset` function. 

### Download data dictionary

In [5]:
setwd("/opt/notebooks")
dataset <- glue("{projectid}:{recordid}")
cmd <- glue("dx extract_dataset {dataset} -ddd")
system(cmd)
dict_files <- list.files(pattern="codings|data_dictionary|entity_dictionary")
data_dict_file <- dict_files[grepl("data_dictionary", dict_files)]

#### Data dictionary filter function

In [6]:
#' @title filter_data_dict
#'
#' @param dict_path, str, path to the dataset.data_dictionary.csv
#' @param codes_str, list, list of lists representing UKBB column name, table entity, and search strategy list(name=, entity=, search=). 
#'   name must be a valid column name in the data_dictionary, entity a valid entity in the entity dictionary, and search either "matches"
#'   for exact matches, or starts with to match cases of multiple instances (repeated measures usually)
#'
#' @returns a filtered subset of the data_dictionary 
#'
filter_data_dict <- function(dict_path, codes_struc) {
    
    data_dict <- fread(dict_path)
    
    d <- lapply(codes_struc, function(x) {
        
        d0 <- data.table()
        if (x$search=="matches") {
            d0 <- data_dict[entity==x$entity & name==x$name]
        } else if (x$search=="startswith") {
            d0 <- data_dict[entity==x$entity & grepl(paste0("^", x$name), name)]
        }
        
        if (nrow(d0)==0) {
            cat(glue("Code [{x$name}] not found in data dictionary\n"))
            stop("Code not found error")
        }
        
        d0
        
    }) |> rbindlist(idcol = "item")
    
    return(d)
}

### Data extraction function

In [7]:
#' @title extract_data
#'
#' @param dataset, str, a valid dataset id - format "{projectid}:{recordid}" 
#' @param fields, str, vector of UK-BB format column names e.g. p31
#' @param entity, str, string of length one - the entity to extract from e.g. participants
#' @param output, str, the base name for the output file, no extension
#'
#' @returns NULL side effect is starting a table-exporter job which outputs the file to /hermes3_data directory in the RAP
#'
extract_data <- function(dataset, fields, entity, output) {
    
    field_str <- paste0('-ifield_names="', fields, '"', collapse=" ") 
    
    cmd <- glue(
      "dx run table-exporter ",
      "-idataset_or_cohort_or_dashboard={dataset} ",
      "-ioutput={output} ",
      "-ioutput_format=TSV ",
      "-iheader_style=FIELD-NAME ",
      "{field_str} ",
      "-ientity={entity} ",
      "--destination hermes3_data/"
    )    

    o <- system(cmd, intern = TRUE)
    cat(o, sep = "\n")
}

### Define participant data

In [8]:
participant_codes= list(eid                = list(name="eid",       entity="participant", search="matches"),
                        reason_lost_fu     = list(name="p190",      entity="participant", search="matches"),
                        sex                = list(name="p31",       entity="participant", search="matches"),
                        age                = list(name="p21022",    entity="participant", search="matches"),
                        ethnicity          = list(name="p21000",    entity="participant", search="startswith"),
                        genetic_sex        = list(name="p22001",    entity="participant", search="matches"),
                        genetic_ethnicity  = list(name="p22006",    entity="participant", search="matches"),
                        pc1                = list(name="p22009_a1", entity="participant", search="matches"),
                        pc2                = list(name="p22009_a2", entity="participant", search="matches"),
                        pc3                = list(name="p22009_a3", entity="participant", search="matches"),
                        pc4                = list(name="p22009_a4", entity="participant", search="matches"),
                        pc5                = list(name="p22009_a5", entity="participant", search="matches"))

participant_data_dict = filter_data_dict(data_dict_file, participant_codes)
head(participant_data_dict, 3)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,participant,eid,string,global,,NA,NA,Participant Information,,,,NA,,,Participant ID,
reason_lost_fu,participant,p190,integer,,data_coding_1965,NA,NA,Population characteristics > Ongoing characteristics,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=190,NA,,,Reason lost to follow-up,
sex,participant,p31,integer,,data_coding_9,NA,NA,Population characteristics > Baseline characteristics,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=31,NA,,,Sex,


### Define self-report illness data
Give there are lots of columns and extraction fails with so much data I do this in a loop. 

In [12]:
self_illness_codes=list(eid                = list(name="eid",    entity="participant", search="matches"),
                        self_rep_ill       = list(name="p20002", entity="participant", search="startswith"), # 0:3 instances
                        self_rep_ill_year  = list(name="p20008", entity="participant", search="startswith"), # 0:3 instances
                        self_rep_proc      = list(name="p20004", entity="participant", search="startswith"), # 0:3 instances
                        self_rep_proc_year = list(name="p20010", entity="participant", search="startswith")) # 0:3 instances

self_rep_data_dict = filter_data_dict(data_dict_file, self_illness_codes)
head(self_rep_data_dict, 3)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,participant,eid,string,global,,NA,NA,Participant Information,,,,NA,,,Participant ID,
self_rep_ill,participant,p20002_i0_a0,integer,,data_coding_6,NA,NA,Assessment centre > Verbal interview > Medical conditions,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=20002,NA,,,"Non-cancer illness code, self-reported | Instance 0 | Array 0",
self_rep_ill,participant,p20002_i0_a1,integer,,data_coding_6,NA,NA,Assessment centre > Verbal interview > Medical conditions,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=20002,NA,,,"Non-cancer illness code, self-reported | Instance 0 | Array 1",


### Define HES inpatient data

In [13]:
hesin_to_extract = list(eid                = list(name="eid",       entity="hesin", search="matches"),
                        ins_index          = list(name="ins_index", entity="hesin", search="matches"),
                        epistart           = list(name="epistart",  entity="hesin", search="matches"),
                        admidate           = list(name="admidate",  entity="hesin", search="matches"))

hes_data_dict = filter_data_dict(data_dict_file, hesin_to_extract)
head(hes_data_dict, 4)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,hesin,eid,string,,,NA,NA,,,,,NA,participant:eid,many_to_one,Participant ID,
ins_index,hesin,ins_index,integer,,,NA,NA,,,,,NA,,,Instance index,
epistart,hesin,epistart,date,,,NA,NA,,,,,NA,,,Episode start date,
admidate,hesin,admidate,date,,,NA,NA,,,,,NA,,,Date of admission to hospital,


### Define HES diagnoses data

In [14]:
hesdiag_to_extract=list(eid                = list(name="eid",        entity="hesin_diag", search="matches"),
                        ins_index          = list(name="ins_index",  entity="hesin_diag", search="matches"),
                        epistart           = list(name="diag_icd9",  entity="hesin_diag", search="matches"),
                        admidate           = list(name="diag_icd10", entity="hesin_diag", search="matches"))

hesdiag_data_dict = filter_data_dict(data_dict_file, hesdiag_to_extract)
head(hesdiag_data_dict, 4)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,hesin_diag,eid,string,,,NA,NA,,,,,NA,,,Participant ID,
ins_index,hesin_diag,ins_index,integer,,,NA,NA,,,,,NA,,,Instance index,
epistart,hesin_diag,diag_icd9,string,,data_coding_87,NA,NA,,,,,NA,,,Diagnoses - ICD9,
admidate,hesin_diag,diag_icd10,string,,data_coding_19,NA,NA,,,,,NA,,,Diagnoses - ICD10,


### Define HES procedures data

In [15]:
hesproc_to_extract=list(eid                = list(name="eid",       entity="hesin_oper", search="matches"),
                        ins_index          = list(name="ins_index", entity="hesin_oper", search="matches"),
                        epistart           = list(name="oper3",     entity="hesin_oper", search="matches"),
                        admidate           = list(name="oper4",     entity="hesin_oper", search="matches"))
hesoper_data_dict = filter_data_dict(data_dict_file, hesproc_to_extract)
head(hesoper_data_dict, 4)

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,hesin_oper,eid,string,,,NA,NA,,,,,NA,,,Participant ID,
ins_index,hesin_oper,ins_index,integer,,,NA,NA,,,,,NA,,,Instance index,
epistart,hesin_oper,oper3,string,,data_coding_259,NA,NA,,,,,NA,,,Operative procedures - OPCS3,
admidate,hesin_oper,oper4,string,,data_coding_240,NA,NA,,,,,NA,,,Operative procedures - OPCS4,


## Run Table-Exporter extraction

In [21]:
data_files <- list(
    demog = "/mnt/project/hermes3_data/data_participant.tsv",
    self  = "/mnt/project/hermes3_data/data_selfreportedillness.tsv",
    hesin = "/mnt/project/hermes3_data/data_hesin.tsv",
    diag  = "/mnt/project/hermes3_data/data_hesin_diag.tsv",
    oper  = "/mnt/project/hermes3_data/data_hesin_oper.tsv"
)

if (!file.exists(data_files$demog)) {
    extract_data(dataset=dataset, fields=participant_data_dict$name, entity="participant", output = "data_participant")
}
if (!file.exists(data_files$self)) {
    extract_data(dataset=dataset, fields=self_rep_data_dict$name,    entity="participant", output = "data_selfreportedillness")
}
if (!file.exists(data_files$hesin)) {
    extract_data(dataset=dataset, fields=hes_data_dict$name,         entity="hesin",       output = "data_hesin")
}
if (!file.exists(data_files$diag)) {
    extract_data(dataset=dataset, fields=hesdiag_data_dict$name,     entity="hesin_diag",  output = "data_hesin_diag")
}
if (!file.exists(data_files$oper)) {
    extract_data(dataset=dataset, fields=hesoper_data_dict$name,     entity="hesin_oper",  output = "data_hesin_oper")
}

## Read in extracted data

In [22]:
counter <- 0
while(TRUE) {
    
    if (counter > 60) {
        cat("Waited 1 hour and files not extracted - aborting\n")
        stop("extract timeout error")
    }
    
    found <- sapply(data_files, file.exists)
    
    if (!all(found)) {
        cat(glue('****\nWaiting for extraction - {counter} minutes elapsed\n****\n'), sep="\n")
        cat(glue('{ifelse(found,"","->")}{names(found)}: file_found={found} : {data_files}'), sep="\n")
        flush.console()
        Sys.sleep(60*5)
        counter <- counter + 1
    } else {
        cat(glue('Reading data files\n'))
        flush.console()
        data_files <- lapply(data_files, fread, nrow=5)
        break
    }
}

Reading data files

### Rename columns

In [26]:
data_files

$demog
       eid 190-0.0 31-0.0 21022-0.0 21000-0.0 21000-1.0 21000-2.0 21000-3.0
     <int>  <lgcl> <char>     <int>    <char>    <lgcl>    <lgcl>    <lgcl>
1: 1062757      NA Female        62   British        NA        NA        NA
2: 2217356      NA   Male        57   British        NA        NA        NA
3: 3712401      NA   Male        56   British        NA        NA        NA
4: 1011090      NA Female        49   British        NA        NA        NA
5: 2874739      NA Female        59   British        NA        NA        NA
   22001-0.0 22006-0.0 22009-0.1 22009-0.2 22009-0.3 22009-0.4 22009-0.5
      <char>    <char>     <num>     <num>     <num>     <num>     <num>
1:    Female Caucasian  -12.6693   4.30928 -2.234380 -0.109226  -7.36400
2:                            NA        NA        NA        NA        NA
3:      Male Caucasian  -13.7558   5.64773 -3.581870  7.741650  21.02850
4:    Female Caucasian  -12.3794   2.03865 -0.837131 -0.562303   2.60283
5:    Female Caucasian  -12.6667   4.18819 -1.969960  2.858750   0.65970

$self
       eid                20002-0.0 20002-0.1      20002-0.2 20002-0.3
     <int>                   <char>    <char>         <char>    <lgcl>
1: 1000074         uterine fibroids                                 NA
2: 1000194 subarachnoid haemorrhage                                 NA
3: 1000258             hypertension  migraine osteoarthritis        NA
4: 1000280 headaches (not migraine)                                 NA
5: 1000299 irritable bowel syndrome                                 NA
   20002-0.4 20002-0.5 20002-0.6 20002-0.7 20002-0.8 20002-0.9 20002-0.10
      <lgcl>    <lgcl>    <lgcl>    <lgcl>    <lgcl>    <lgcl>     <lgcl>
1:        NA        NA        NA        NA        NA        NA         NA
2:        NA        NA        NA        NA        NA        NA         NA
3:        NA        NA        NA        NA        NA        NA         NA
4:        NA        NA        NA        NA        NA        NA         NA
5:        NA        NA        NA        NA        NA        NA         NA
   20002-0.11 20002-0.12 20002-0.13 20002-0.14 20002-0.15 20002-0.16 20002-0.17
       <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>
1:         NA         NA         NA         NA         NA         NA         NA
2:         NA         NA         NA         NA         NA         NA         NA
3:         NA         NA         NA         NA         NA         NA         NA
4:         NA         NA         NA         NA         NA         NA         NA
5:         NA         NA         NA         NA         NA         NA         NA
   20002-0.18 20002-0.19 20002-0.20 20002-0.21 20002-0.22 20002-0.23 20002-0.24
       <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>
1:         NA         NA         NA         NA         NA         NA         NA
2:         NA         NA         NA         NA         NA         NA         NA
3:         NA         NA         NA         NA         NA         NA         NA
4:         NA         NA         NA         NA         NA         NA         NA
5:         NA         NA         NA         NA         NA         NA         NA
   20002-0.25 20002-0.26 20002-0.27 20002-0.28 20002-0.29 20002-0.30 20002-0.31
       <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>     <lgcl>
1:         NA         NA         NA         NA         NA         NA         NA
2:         NA         NA         NA         NA         NA         NA         NA
3:         NA         NA         NA         NA         NA         NA         NA
4:         NA         NA         NA         NA         NA         NA         NA
5:         NA         NA         NA         NA         NA         NA         NA
   20002-0.32 20002-0.33 20002-1.0 20002-1.1 20002-1.2 20002-1.3 20002-1.4
       <lgcl>     <lgcl>    <lgcl>    <lgcl>    <lgcl>    <lgcl>    <lgcl>
1:         NA         NA        NA        NA        NA        NA        NA
2:         NA         N

In [25]:
rename_cols <- function(d, code_struc) {
    for (col in names(code_struc)) {
        if (code_struc[[col]]$search=="matches") {
            setnames(d, code_struc[[col]]$name, col)
        } else if (code_struc[[col]]$search=="startswith") {
            regex     <- paste0("^", code_struc[[col]]$name)
            matches   <- names(d)[grepl(regex, names(d))]
            new_names <- paste0(col, "_", 1:length(matches))
            setnames(d, matches, new_names)
        }
    }
    return(d)
}

data_files$demog <- rename_cols(data_files$demog, code_struc=participant_codes)
data_files$self  <- rename_cols(data_files$self,  code_struc=self_illness_codess)
data_files$hesin <- rename_cols(data_files$hesin, code_struc=hesin_to_extract)
data_files$diag  <- rename_cols(data_files$diag,  code_struc=hesdiag_to_extract)
data_files$oper  <- rename_cols(data_files$oper,  code_struc=hesproc_to_extract)

data_files

ERROR: Error in setnames(d, code_struc[[col]]$name, col): Items of 'old' not found in column names: [p190]. Consider skip_absent=TRUE.


## Data processing

### Read in the heRmes codes
ICD-9/10 coding is provided but we need to add the self reported codes from the UK-BB too.

In [ ]:
codes <- fread(file.path("heRmes", "inst", "extdata", "hermes_3_codes", "hermes_3_codes.tsv"))
self_reported_codes <- list(
  list(name = "Heart Failure",                      code = "1076", code_type = "ukbb_self_reported_illness"),
  list(name = "Myocardial infarction",              code = "1075", code_type = "ukbb_self_reported_illness"),
  list(name = "Hypertrophic cardiomyopathy",        code = "1588", code_type = "ukbb_self_reported_illness"),
  list(name = "Coronary artery bypass grafting",    code = "1095", code_type = "ukbb_self_reported_procedure"),
  list(name = "Percutaneous coronary intervention", code = "1070", code_type = "ukbb_self_reported_procedure")
)
codes <- rbind(codes,
               data.table(Concept     = paste0(sapply(self_reported_codes, function(x) x$name), " Self Reported"),
                          Code        = sapply(self_reported_codes, function(x) x$code),
                          Source      = sapply(self_reported_codes, function(x) x$code_type),
                          Description = sapply(self_reported_codes, function(x) x$name)))
codes[, `:=`(code      = Code,
             code_type = fcase(Source=="ICD10", "icd10",
                               Source=="ICD9",  "icd9",
                               Source=="OPCS4", "opcs4",
                               Source=="ukbb_self_reported_illness", "ukbb_self_reported_illness",
                               Source=="ukbb_self_reported_procedure", "ukbb_self_reported_procedure"))]
codes <- codes[!is.na(code_type)]                                               

### Clean up the cohort data

In [86]:
ethnicity_codes <- list(
  white                 = 1,
  british               = 1001,
  white_black_caribbean =	2001,
  indian                = 3001,
  caribbean             = 4001,
  mixed                 = 2,
  irish                 =	1002,
  white_black_african	  = 2002,
  pakistani             = 3002,
  african	              = 4002,
  asian_or_asian_british=	3,
  any_other_white       =	1003,
  white_asian           =	2003,
  bangladeshi           =	3003,
  any_other_black       =	4003,
  black_or_black_british=	4,
  any_other_mixed       =	2004,
  any_other_asian       =	3004,
  chinese               =	5,
  other_ethnic_group    = 6)

data_files$demog <- data_files$demog[, 
    list(eid               = eid,
         age               = as.integer(age),
         sex               = factor(sex, levels = 0:1, labels = c("female", "male")),
         ethnicity         = factor(ethnicity, levels = unlist(ethnicity_codes), labels = names(ethnicity_codes)),
         ethnicity_group   = factor(sub("([0-9])00[0-9]", "\\1", ethnicity), levels = unlist(ethnicity_codes), labels = names(ethnicity_codes)),
         genetic_sex       = factor(genetic_sex, levels = 0:1, labels = c("female", "male")),
         genetic_ethnicity = factor(genetic_ethnicity, levels = 1, labels = c("caucasian")))]

# check
stopifnot("Failed to parse some date of births" = all(!is.na(demog$dob)))
stopifnot("some ages / dob indicate cohort age <37, is this right?" = all(demog$dob <= as.Date("1972-01-01")))

ERROR: Error in eval(jsub, SDenv, parent.frame()): object 'ethnicity' not found


### Self-report illness codes to long

In [ ]:
self_rep_code_regex <- "20002-[0-9]+\\.[0-9]+"
self_rep_year_regex <- "20008-[0-9]+\\.[0-9]+"
self_rep_code_cols <- grep(self_rep_code_regex, names(data_files$self), value = TRUE)
self_rep_year_cols <- grep(self_rep_year_regex, names(data_files$self), value = TRUE)
data_files$self[, (self_rep_code_cols) := lapply(.SD, as.character), .SDcols = self_rep_code_cols]
data_files$self[, (self_rep_year_cols) := lapply(.SD, as.numeric),   .SDcols = self_rep_year_cols]
data_files$self_illness <- data.table::melt(data_files$self,
                                            id.vars = "eid",
                                            measure = patterns(self_rep_code_regex, self_rep_year_regex),
                                            variable.name = "element",
                                            value.name = c("code", "year"),
                                            na.rm = TRUE)
data_files$self_illness <- data_files$self_illness[year != -1 & year != -3] # unknown / prefer not to answer
data_files$self_illness[, `:=`(date      = lubridate::ymd(paste0(as.character(floor(year)), "-01-01")) + lubridate::days(as.integer(365.25 * (year - floor(year)))),
                               year      = NULL,
                               element   = NULL,
                               code      = as.character(code),
                               code_type = "ukbb_self_reported_illness")]

# check self report illness table
stopifnot("unable to parse dates for self-reported illness codes" = all(!is.na(data_files$self_illness$date)))
stopifnot("are you sure something happened before 1900?" = all(data_files$self_illness$date > as.Date("1900-01-01")))

### Self-report procedure codes to long

In [ ]:
self_rep_proc_code_regex <- "20004-[0-9]+\\.[0-9]+"
self_rep_proc_year_regex <- "20010-[0-9]+\\.[0-9]+"
self_rep_proc_code_cols <- grep(self_rep_proc_code_regex, names(data_files$self), value = TRUE)
self_rep_proc_year_cols <- grep(self_rep_proc_year_regex, names(data_files$self), value = TRUE)
data_files$self[, (self_rep_proc_code_cols) := lapply(.SD, as.character), .SDcols = self_rep_proc_code_cols]
data_files$self[, (self_rep_proc_year_cols) := lapply(.SD, as.numeric),   .SDcols = self_rep_proc_year_cols]
data_files$self_oper <- data.table::melt(self_oper,
                                         id.vars = "eid",
                                         measure = patterns(self_rep_proc_code_regex, self_rep_proc_year_regex),
                                         variable.name = "element",
                                         value.name = c("code", "year"),
                                         na.rm = TRUE)
data_files$self_oper <- data_files$self_oper[year != -1 & year != -3] # unknown / prefer not to answer
data_files$self_oper[, `:=`(date      = lubridate::ymd(paste0(as.character(floor(year)), "-01-01")) + lubridate::days(as.integer(365.25 * (year - floor(year)))),
                            year      = NULL,
                            element   = NULL,
                            code      = as.character(code),
                            code_type = "ukbb_self_reported_procedure")]

# check self report illness table
stopifnot("unable to parse dates for self-reported procedure codes" = all(!is.na(data_files$self_oper$date)))
stopifnot("are you sure something happened before 1900?" = all(data_files$self_oper$date > as.Date("1900-01-01")))

### Inpatient diagnosis codes

In [ ]:
data_files$hesin[is.na(epistart) | epistart == "", epistart := admidate]
data_files$diag[data_files$hesin, date := as.Date(i.epistart), on = c("eid", "ins_index")]
data_files$diag[diag_icd9 == "", diag_icd9 := NA_character_]
data_files$diag[diag_icd10 == "", diag_icd10 := NA_character_]
data_files$diag <- data.table::melt(data_files$diag,
                                    id.vars = c("eid", "date"),
                                    measure.vars  = c("diag_icd9", "diag_icd10"),
                                    variable.name = "code_type",
                                    value.name = "code",
                                    na.rm = TRUE)
data_files$diag[, code_type := data.table::fcase(code_type == "diag_icd9", "icd9",
                                                 code_type == "diag_icd10", "icd10")]

### Inpatient procedure codes

In [ ]:
data_files$oper[data_files$hesin, date := as.Date(i.epistart), on = c("eid", "ins_index")]
data_files$oper[oper3 == "", oper3 := NA_character_]
data_files$oper[oper4 == "", oper4 := NA_character_]
data_files$oper <- data.table::melt(data_files$oper,
                                    id.vars = c("eid", "date"),
                                    measure.vars  = c("oper3", "oper4"),
                                    variable.name = "code_type",
                                    value.name = "code",
                                    na.rm = TRUE)
data_files$oper[, code_type := data.table::fcase(code_type == "oper3", "opcs3",
                                                 code_type == "oper4", "opcs4")]

### Combine all codes
Keep only unique codes per individuals at the code's first occurance.

In [ ]:
combined <- rbind(data_files$selfs_illness, data_files$self_oper, data_files$diag, data_files$oper)
combined <- codes[combined, on = c("code" = "code", "code_type" = "code_type"), allow.cartesian = TRUE]
combined <- combined[!is.na(Concept)]
combined <- combined[combined[, .I[which.min(date)], by = c("eid", "Concept")]$V1]

### Annotate the cohort with the codes

In [ ]:
concepts <- unique(codes$Concept)
for (g in concepts) {

  col_name <- tolower(gsub(" ", "_", gsub("[()]","",g)))
  demog[combined[Concept == g], paste0(col_name, c("", "_first_date")) := list(TRUE, as.Date(i.date)), on = "eid"]
  demog[is.na(get(col_name)), (col_name) := FALSE]

}

### Remove withdrawals

In [ ]:
cohort <- cohort[reason_lost_fu!=5] # 5 - Participant has withdrawn consent for future linkage

## Run phenotyping

In [170]:
# any ischaemic ICD codes
ischaemic_cols <- c("myocardial_infarction", "coronary_artery_bypass_grafting", "percutaneous_coronary_intervention", "thrombolysis_coronary", "ischaemic_cardiomyopathy")
cohort[, ischaemic := rowSums(.SD) > 0, .SDcols = ischaemic_cols]
cohort[, ischaemic_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(ischaemic_cols, "_first_date")]

# combined NICM ICD codes
nicm_cols <- c("dilated_cardiomyopathy", "dilated_cardiomyopathy_associated_with", "left_ventricular_systolic_dysfunction")
cohort[, nicm_comb := rowSums(.SD) > 0, .SDcols = nicm_cols]
cohort[, nicm_comb_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(nicm_cols, "_first_date")]

# any ischaemic self reported codes
self_isch_cols <- c("myocardial_infarction_self_reported", "coronary_artery_bypass_grafting_self_reported", "percutaneous_coronary_intervention_self_reported")
cohort[, self_isch := rowSums(.SD) > 0, .SDcols = self_isch_cols]
cohort[, self_isch_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_isch_cols, "_first_date")]

# any self reported HF codes
self_hf_col = c("heart_failure_self_reported")
cohort[, self_hf := rowSums(.SD) > 0, .SDcols = self_hf_col]
cohort[, self_hf_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_hf_col, "_first_date")]

# any self reported HCM codes
self_hcm_col = c("hypertrophic_cardiomyopathy_self_reported")
cohort[, self_hcm := rowSums(.SD) > 0, .SDcols = self_hcm_col]
cohort[, self_hcm_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_hcm_col, "_first_date")]

# HF exclusions
cohort[, hf_exclude := withdrawal == TRUE |
                       congenital_heart_disease==TRUE |  # all congenital heart disease
                       (heart_failure==FALSE & (self_hf==TRUE | ischaemic==TRUE | self_isch==TRUE)) | # non-HF but with ischaemic ICD history or self reported heart failure or ischaemic history
                       (heart_failure==TRUE  & (ischaemic==TRUE & ischaemic_first_date > heart_failure_first_date))] # HF but with first ischaemic event after the HF diagnosis
# pheno 1
cohort[, pheno1 := congenital_heart_disease==FALSE & heart_failure==TRUE]

# pheno 2
cohort[, pheno2 := hf_exclude==FALSE & heart_failure==TRUE & ischaemic==TRUE]

# pheno 3
cohort[, pheno3 := hf_exclude==FALSE & heart_failure==TRUE & ischaemic==FALSE]

# HF controls
cohort[, hf_control := hf_exclude==FALSE & pheno1==FALSE & pheno2==FALSE & pheno3==FALSE]

# DCM exclusions
cohort[, cm_exclude := withdrawal == TRUE |
                       congenital_heart_disease==TRUE |  # all congenital heart disease
                       hypertrophic_cardiomyopathy==TRUE | self_hcm==TRUE | # all HCM
                       restrictive_cardiomyopathy==TRUE] # all RCM

# pheno 4
cohort[, pheno4 := cm_exclude==FALSE &
                   !(dilated_cardiomyopathy==TRUE  & (ischaemic==TRUE & ischaemic_first_date <= dilated_cardiomyopathy_first_date)) & # DCM but with first ischaemic event prior to the DCM diagnosis
                   dilated_cardiomyopathy==TRUE]

# pheno 5
cohort[, pheno5 := cm_exclude==FALSE &
         !(nicm_comb==TRUE & (ischaemic==TRUE & ischaemic_first_date <= nicm_comb_first_date)) &  # NICM but with first ischaemic event prior to the NICM diagnosis
         nicm_comb==TRUE]

# CM controls
cohort[, cm_control := cm_exclude==FALSE &
                       pheno4==FALSE &
                       pheno5==FALSE &
                       ischaemic==FALSE &
                       self_isch==FALSE]


# check HF phenotyping
base_cols <- c("eid", "age", "sex", "ethnicity", "ethnicity_group","genetic_sex", "genetic_ethnicity", paste0("pc",1:5))
sum_cols <- names(cohort)[!names(cohort) %in% base_cols
                          &
                          !grepl("date", names(cohort))]
summary <- data.table (name = c("total", sum_cols), sex = "all", N = c(nrow(cohort), cohort[, .(sapply(.SD, sum)), .SDcols = sum_cols]$V1))
summary <- rbind(summary,
                 data.table(name = rep(sum_cols, 2), cohort[, .(N = sapply(.SD, sum)), .SDcols = sum_cols, by = "sex"]), fill=TRUE)
summary

# write summary
fwrite(dcast(summary, name ~ sex, value.var = "N"),
       file = "hermes3_phenotype_summary.tsv"),
       sep  = "\t")

cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/1) = ", sum(summary[name%in%c("hf_exclude",paste0("pheno1"),"hf_control"), N]), "\n")
cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/2/3) = ", sum(summary[name%in%c("hf_exclude",paste0("pheno",2:3),"hf_control"), N]), "\n")
cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/5) = ", sum(summary[name%in%c("cm_exclude",paste0("pheno5"),"cm_control"), N]), "\n")

# write out
fwrite(cohort[, mget(c(base_cols[base_cols != "eid"],
                       paste0("pheno", 1:3), "hf_exclude", "hf_control",
                       paste0("pheno", 4:5), "cm_exclude", "cm_control"))],
       file = "hermes3_phenotypes.tsv.gz"),
       sep  = "\t")

ERROR: Error in parse(text = input): <text>:79:47: unexpected ','
78: fwrite(dcast(summary, name ~ sex, value.var = "N"),
79:        file = "hermes3_phenotype_summary.tsv"),
                                                  ^


In [ ]:
# End